In [1]:
# !nvcc --version

In [2]:
# !apt-get --purge remove cuda nvidia* libnvidia-*
# !dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !apt-get remove cuda-*
# !apt autoremove
# !apt-get update

In [3]:
# #Download CUDA 10.0
# !wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
# #install CUDA kit dpkg
# !dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
# !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
# !apt-get update
# !apt-get install cuda-10-0

In [4]:
# !nvcc --version

In [5]:
!pip install gluonts
# !pip install mxnet-cu100mkl
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [6]:
import pandas as pd
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx import Trainer
import numpy as np
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
import mxnet as mx
#df = pd.read_csv('/content/drive/MyDrive/Electricity/LD2011_2014.txt', sep = ';', index_col = 0, dtype=str)
#df.drop(df.tail(1).index,inplace=True)


/usr/local/lib/python3.8/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gluonts/model/deepar/__init__.py:18: FutureWarning: The module gluonts.model.deepar has been moved to gluonts.mx.model.deepar. In GluonTS v0.12 it will be no longer possible to use the old path. Try to use 'from gluonts.mx import DeepAREstimator'.
  warnings.warn(


In [7]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip

--2022-12-17 02:43:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261335609 (249M) [application/x-httpd-php]
Saving to: ‘LD2011_2014.txt.zip’

LD2011_2014.txt.zip 100%[===================>] 249.23M  47.5MB/s    in 5.4s    

2022-12-17 02:44:00 (46.4 MB/s) - ‘LD2011_2014.txt.zip’ saved [261335609/261335609]



In [8]:
!unzip LD2011_2014.txt.zip

Archive:  LD2011_2014.txt.zip
  inflating: LD2011_2014.txt         
   creating: __MACOSX/
  inflating: __MACOSX/._LD2011_2014.txt  


In [9]:
df = pd.read_csv('./LD2011_2014.txt', sep = ';', index_col = 0, dtype=str)
df.drop(df.tail(1).index,inplace=True)

In [10]:
from datetime import datetime
df.index = pd.to_datetime(df.index)
df = df.applymap(lambda x: str(x.replace(',','.')))
df = df.astype(float)
df = df.reset_index()
df = df.groupby([pd.Grouper(key='index', freq='1D')]).sum()
df = df.drop(datetime(2014,12,25))

In [11]:
def cleaning(dataframe, threshold=90):
    dataframe = dataframe.copy(deep=True)
    dataframe = dataframe[dataframe.index.year >= 2012]
    # remove the extra day in leap year
    dataframe = dataframe[dataframe != pd.to_datetime('20120229')]

    # find users that are active since 2012
    # active_user = [user for user in dataframe.user.unique() if dataframe[(dataframe.user==user) * (dataframe.date==pd.to_datetime('20120101'))].sum_per_day.item() > 0]
    # dataframe = dataframe[dataframe.user.isin(active_user)]

    def without_leading_consecutive_zeros(dataframe, threshold, by="user", col="sum_per_day"):
        # reference: https://datascience.stackexchange.com/questions/20587/find-the-consecutive-zeros-in-a-dataframe-and-do-a-conditional-replacement
        group = dataframe.groupby(by=by)[col].apply(lambda x: (~((x == x.shift()) & (x == 0))).cumsum())
        mask = dataframe[col].groupby(group).transform('count').lt(threshold)
        mask |= (group != 1)
        return mask

    # mask = without_leading_consecutive_zeros(dataframe, threshold)
    # print(f'dropping {(~mask).sum()} rows with leading zeros...')
    # dataframe = dataframe.loc[mask, :]

    # count 0 usage days
    zero_count = dataframe.apply(lambda x: np.sum(x == 0))
    abnormal = zero_count[zero_count > 370]
    print(abnormal)

    # sns.relplot(data=dataframe[dataframe.user.isin(abnormal.index)], 
    #         x="date", y="sum_per_day", row="user",
    #         kind="line", aspect=4, facet_kws={'sharey': False})

    dataframe = dataframe.drop(columns=abnormal.index.tolist())
    

    return dataframe

df = cleaning(df)

MT_012    606
MT_015    629
MT_030    494
MT_039    641
MT_041    621
MT_092    396
MT_106    744
MT_107    744
MT_108    744
MT_109    779
MT_110    744
MT_111    744
MT_112    773
MT_113    744
MT_115    744
MT_116    779
MT_117    744
MT_120    744
MT_121    744
MT_122    744
MT_133    802
MT_160    765
MT_165    613
MT_170    424
MT_178    929
MT_179    719
MT_181    794
MT_185    537
MT_186    634
MT_223    828
MT_224    540
MT_289    698
MT_305    708
MT_322    607
MT_337    747
MT_347    621
dtype: int64


In [12]:
## Reset the index as we do not need dates for DeepAR and also transpose the data 
df_input=df.reset_index(drop=True).T.reset_index()

## Converting hosueholds into category encoding to feed into the model 
ts_code=df_input["index"].astype('category').cat.codes.values

In [13]:
def train_test_split(df):
  df_trainval=df_input.iloc[:,1:-270]
  df_test=df_input.iloc[:,-270:]

  return df_trainval,df_test

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm.autonotebook import tqdm
from gluonts.evaluation import Evaluator
import pickle

df_trainval = train_test_split(df_input)[0]
df_test = train_test_split(df_input)[1].values

freq="D"
start_train = pd.Timestamp("2011-01-01", freq=freq)
start_test = start_train + pd.Timedelta(days=len(df_trainval.iloc[0])) 
def model(df_train, freq, prediction_length, context_length, num_layers, num_cells, epochs, learning_rate):
  train_values = df_train.values
  train_ds = ListDataset([
      {
          FieldName.TARGET: target,
          FieldName.START: start_train,
          FieldName.FEAT_STATIC_CAT: fsc
      }
      for (target, fsc) in zip(train_values,
                              ts_code.reshape(-1,1))
  ], freq=freq)

  estimator = DeepAREstimator(freq=freq, 
                              context_length=context_length,
                              prediction_length=prediction_length,
                              use_feat_static_cat=True,
                              cardinality=[1],
                              num_layers=num_layers,
                              num_cells=num_cells,
                              cell_type='lstm',
                              trainer= Trainer(ctx=mx.context.cpu(), 
                              learning_rate=learning_rate,
                              epochs=epochs))
                  
  predictor = estimator.train(training_data=train_ds)

  return predictor

df_train = df_trainval.iloc[:,:-270]
df_val = df_trainval.iloc[:,-270:]

start_val = start_train + pd.Timedelta(days=len(df_train.columns))

context_length=90
num_layers=3
num_cells=30
epochs=20
learning_rate=0.01
batch_size=128
predictor1 = model(df_train, freq, 90, context_length, num_layers, num_cells, epochs, learning_rate)
predictor2 = model(df_train, freq, 180, context_length, num_layers, num_cells, epochs, learning_rate)
predictor3 = model(df_train, freq, 270, context_length, num_layers, num_cells, epochs, learning_rate)

predictors = [predictor1, predictor2, predictor3]

In [ ]:
# 90 days
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_test,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_test[:90],
                            ts_code.reshape(-1,1))
], freq=freq)

forecast_it_test_90, ts_it_test_90 = make_evaluation_predictions(
    dataset=test_ds,  
    predictor=predictors[0],  
    num_samples=20
)

# print("Obtaining time series conditioning values ...")
tss_test= list(tqdm(ts_it_test_90, total=len(df_test[:90])))
# print("Obtaining time series predictions ...")
forecasts_test = list(tqdm(forecast_it_test_90, total=len(df_test[:90])))

agg_metrics_test_90, item_metrics_test_90 = evaluator(tss_test, forecasts_test)
mape_test = agg_metrics_test_90['MAPE']
mape_test

In [35]:
#180 days
test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_test,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_test[:180],
                            ts_code.reshape(-1,1))
], freq=freq)

forecast_it_test_180, ts_it_test_180 = make_evaluation_predictions(
    dataset=test_ds,  
    predictor=predictors[1],  
    num_samples=20
)

# print("Obtaining time series conditioning values ...")
tss_test= list(tqdm(ts_it_test_180, total=len(df_test[:180])))
# print("Obtaining time series predictions ...")
forecasts_test = list(tqdm(forecast_it_test_180, total=len(df_test[:180])))

agg_metrics_test_180, item_metrics_test_180 = evaluator(tss_test, forecasts_test)
mape_test = agg_metrics_test_180['MAPE']
mape_test

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

Running evaluation: 180it [00:00, 2203.12it/s]


1.2811341113808714

In [36]:
#270 days
test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_test,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_test[:270],
                            ts_code.reshape(-1,1))
], freq=freq)

forecast_it_test_270, ts_it_test_270 = make_evaluation_predictions(
    dataset=test_ds,  
    predictor=predictors[2],  
    num_samples=20
)

# print("Obtaining time series conditioning values ...")
tss_test= list(tqdm(ts_it_test_270, total=len(df_test[:270])))
# print("Obtaining time series predictions ...")
forecasts_test = list(tqdm(forecast_it_test_270, total=len(df_test[:270])))

agg_metrics_test_270, item_metrics_test_270 = evaluator(tss_test, forecasts_test)
mape_test = agg_metrics_test_270['MAPE']
mape_test

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

Running evaluation: 270it [00:00, 2155.28it/s]
/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


1.0